In [ ]:
%pip install openai
%pip install pandas
%pip install numpy
%pip install nltk
%pip install gensim

In [1]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key= "28f146892bb54fd3a8ed90c8f7080f0a",
    api_version="2023-12-01-preview",
    azure_endpoint= "https://assignments.openai.azure.com/"
)

In [2]:
#Pre process the text

import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
import numpy

from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "html.parser").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [11]:
def ReadAndGenerateEmbeddings(client, fileName):
    #read from text file
    with open(fileName, 'r') as file:
        data = file.read().replace('\n', ',')

    return GetEmbeddings(client, data)

def GetEmbeddings(client, data):
    # data = clean_text(data)   
    return GenerateEmbedding(client, data)

def GenerateEmbedding(client, data):
    embedding = client.embeddings.create(
        model="text-embedding-ada-002",
        input= data
    )
    return np.array(embedding.data[0].embedding)

In [12]:
villash = ReadAndGenerateEmbeddings(client, "villash.txt")

sagar = ReadAndGenerateEmbeddings(client, "sagar.txt")

abhay = ReadAndGenerateEmbeddings(client, "abhay.txt")

# vidhu = ReadAndGenerateEmbeddings(client, "vidhu.txt")

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 8191 tokens, however you requested 10499 tokens (10499 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [9]:
exampleText = [
    "Add Accessibity tags in pdf",
    # "[ATM BulkSync] Add new orchestrator for AccessSync",
    # "Add Basline Api's",
    # "Increase Cloudporj limits to 5000"
]

exampleEmbedding = GetEmbeddings(client, exampleText)

In [10]:
exampleEmbedding.size

1536

In [ ]:
def GetSimilarityScore(embedding1, embedding2):
    return np.dot(embedding1, embedding2) / (np.linalg.norm(embedding1) * np.linalg.norm(embedding2))


#print the similarity score along with name
print(GetSimilarityScore(villash, exampleEmbedding))
print(GetSimilarityScore(sagar, exampleEmbedding))
# print(GetSimilarityScore(vidhu, exampleEmbedding))
print(GetSimilarityScore(abhay, exampleEmbedding))
